In [20]:
# Try shifting Uniprot and merging with ProIntVar files -- check sequences
# Do Prediction + sequence

import os
import pandas as pd
import numpy as np
import glob

os.chdir('/homes/2524591/data')

data = pd.read_csv('merged_final.csv')



for i,row in data.iterrows():  
    data_uni = []
    pred = []
    
    pdb_id = row.iloc[2] 
    uniprot_seq = row.iloc[6]
#     print(row)
    AF_pred = row.iloc[7]
    scop = row.iloc[1]
    
    # Open Merged file - some pdb id's missing
 
    try:
        df3 = pd.read_csv('/homes/2524591/data/prointvar_col/%s.csv' % pdb_id)
    except FileNotFoundError:
        print(f'{pdb_id} is missing')
        continue
        
        
    df3_null = df3.replace('null', np.nan, regex=True)
#     df3_n = df3_null['UniProt_dbResNum'].replace('', np.nan, inplace=True)
    df3_val = df3_null.dropna()
    
# #     print(df3_val)
    
    try:
        chain = scop[-2]
        chain_cap = chain.upper() 
        try:
            df_chain = df3_val.loc[df3_val['PDB_dbChainId'].str.startswith(chain_cap)]
        except AttributeError:
            print(f'Invalid Chain data: {pdb_id}')
    except IndexError:
        print(f'SCOPe ID not valid: PDB_ID = {pdb_id}')
        continue

    uniprot_offset = df_chain['UniProt_dbResNum'] # Add chain code
    df_u = pd.DataFrame(uniprot_offset)
    df_drop = df_u.dropna()



    for row in df_drop.iterrows():
        position_pdb = row[1][0]
        try:
            pos = position_pdb.astype(int)
        except ValueError:
            print(f'Skipping {pdb_id}: PDB - Invalid data...')
            continue        

        if pos < len(uniprot_seq) and pos < len(AF_pred):
            shifted_seq = uniprot_seq[pos-1]
            shifted_pred = AF_pred[pos-1]
            data_uni.append(shifted_seq)
            pred.append(shifted_pred)

    
    df_d = pd.DataFrame(data_uni)
    df_pr = pd.DataFrame(pred)     
    
    val_row = df_chain['UniProt_dbResNum'].first_valid_index()
    df3_shift = df_chain.iloc[val_row:]
    try:
        df_uni = df_d.shift(periods=val_row, fill_value=0)
        df_pr_2 = df_pr.shift(periods=val_row, fill_value=0)
    except TypeError:
        print(f'wtf: {pdb_id}')
#     df_pr_2 = df_pr.shift(periods=val_row, fill_value=0)
    
    df3_shift['shifted Uni'] = df_uni    
    df3_shift['AF_predictions'] = df_pr_2
    
    
    
    if os.path.exists(f'/homes/2524591/data/AF_Pro/{pdb_id}.csv'):
        if os.path.exists(f'/homes/2524591/data/AF_Pro/{pdb_id}_2.csv'):
            if os.path.exists(f'/homes/2524591/data/AF_Pro/{pdb_id}_3.csv'):
                if os.path.exists(f'/homes/2524591/data/AF_Pro/{pdb_id}_4.csv'):
                    print(f'Too many!! {pdb_id}')
                else:
                    df = pd.read_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}_3.csv')
                    if not df3_shift.equals(df):
                        try:
                            df3_shift.to_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}_4.csv')
                        
                        except ValueError:
                            print(f'Something wrong with: {pdb_id}')
                            continue
#                     break
            else:
                df = pd.read_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}_2.csv')
                if not df3_shift.equals(df):
                    try:
                        df3_shift.to_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}_3.csv')
                    except ValueError:
                        print(f'Something wrong with: {pdb_id}')
                        continue
                    
#                 break
                    
        else:
            df = pd.read_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}.csv')
            if not df3_shift.equals(df):
                try:
                    df3_shift.to_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}_2.csv')    
                except ValueError:
                    print(f'Something wrong with: {pdb_id}')
                    continue
#             break
    else:
        try:
            df3_shift.to_csv(f'/homes/2524591/data/AF_Pro/{pdb_id}.csv')
        except ValueError:
            print(f'Something wrong with: {pdb_id}')
            continue
#         break
    

/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

Too many!! 1ppj
Too many!! 1ppj


/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2


Too many!! 1ppj


/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2


...

/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

Something wrong with: 1nvu
3pdf is missing


/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

Too many!! 2fyu
Too many!! 2fyu
Too many!! 2fyu
Too many!! 2fyu
Too many!! 2fyu
Too many!! 2fyu


/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['shifted Uni'] = df_uni
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_shift['AF_predictions'] = df_pr_2
/tmp/521538.1.all.q/ipykernel_24197/214337897.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [3]:
# Check no. of files - merged, with seq, with AF_pred
import os
import glob

os.chdir('/homes/2524591/data')


# All merged files from ProIntVar
x = glob.glob('prointvar_col/*.csv')

# # with AF SS prediction and sequence
# y = glob.glob('shift_merged/merged_w_AF/*.csv')


# # with Uniprot sequence -- extra (not needed)
# z = glob.glob('shift_merged/add_shifted_uni_seq/*.csv')

# print(len(x))
# print(len(y))
# print(len(z))

# Save multiple versions of pdb's (all of them -- not just a single copy)
y = glob.glob('AF_Pro/*.csv')

print(len(x))
print(len(y))



877
1099
